# Training and Tuning at Scale

#### Amazon SageMaker provides managed distributed training and tuning capabilities to improve training efficiency, and capabilities to organize and track ML experiments at scale. SageMaker enables techniques such as streaming data into algorithms by using pipe mode for training with data at scale and Managed Spot Training for reduced training costs. 

## ML training at scale with SageMaker distributed libraries

#### Two common scale challenges with ML projects are scaling training data and scaling model size. While increased training data volume, model size, and complexity can potentially result in a more accurate model, there is a limit to the data volume and the model size that you can use with a single compute node, CPU, or GPU. Increased training data volumes and model sizes typically result in more computations, and therefore training jobs take longer to finish, even when using powerful compute instances such as Amazon Elastic Compute Cloud (EC2) p3 and p4 instances.

#### Distributed training is a commonly used technique to speed up training when dealing with scale challenges. Training load can be distributed either across multiple compute instances (nodes), or across multiple CPUs and GPUs (devices) on a single compute instance. There are two strategies for distributed training – data parallelism and model parallelism. Their names are a good indication of what is involved with each strategy. With data parallelism, the training data is split up across multiple nodes (or devices). With model parallelism, the model is split up across the nodes (or devices).

#### Mixed-precision training is a popular technique to handle training at scale and reduce training time. Typically used on compute instances equipped with NVIDIA GPUs, mixed-precision training converts network weights from FP32 representation to FP16, calculates the gradients, converts weights back to FP32, multiplies by the learning rate, and finally updates the optimizer weights.

#### In the data parallelism distribution strategy, the ML algorithm or the neural network-based model is replicated on all devices, and each device processes a batch of data. Results from all devices are then combined. In the model parallelism distribution strategy, the model (which is the neural network) is split up across the devices. Batches of training data are sent to all devices so that the data can be processed by all parts of the model. 

